**Block 1: Environment & Dependency Setup**  
Initialization: Prepares the cloud workspace by installing the Gradio web framework and importing TensorFlow.  
Verification: Confirms NVIDIA T4 GPU availability, ensuring the system can handle the heavy mathematical computations required for deep neural network training.


In [1]:
!pip install gradio -q

import tensorflow as tf
import gradio as gr
import numpy as np
from tensorflow.keras import layers, models

# Verify GPU is active
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
#SUPRESSING ERRORS: -qqq makes it extremely quiet; 2>/dev/null hides the red error/conflict text
!pip install gradio -qqq 2>/dev/null


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.0/103.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.3/553.3 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.3/108.3 kB 9.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tokenizers 0.21.0 requires huggingface-

**Block 2: Data Engineering & Pipeline**  
This block links the Agricultural Pests Dataset and uses `image_dataset_from_directory` to automatically label the 12 pest classes.  
It splits the 6,000 images into training and validation sets while resizing them to 224 × 224 for model compatibility.


In [2]:
DATA_DIR = "/kaggle/input/agricultural-pests-image-dataset"

train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=64  # GPU T4 handles larger batches well
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=64
)

class_names = train_ds.class_names


Found 5494 files belonging to 12 classes.
Using 4396 files for training.
Found 5494 files belonging to 12 classes.
Using 1098 files for validation.


**Block 3: CNN Training & Transfer Learning**  
This block builds a Sequential model using a pre-trained MobileNetV2 base for high-speed feature extraction.  
It compiles the model with the Adam optimizer and trains for 10 epochs, enabling the AI to learn pest patterns with high accuracy on GPU hardware.

In [3]:
# MobileNetV2 is optimized for speed on GPUs
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3), 
    include_top=False, 
    weights='imagenet'
)
base_model.trainable = False 

model = models.Sequential([
    layers.Rescaling(1./255, input_shape=(224, 224, 3)),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(len(class_names), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train for 10 epochs (T4 GPU makes this very fast)
model.fit(train_ds, validation_data=val_ds, epochs=10)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


69/69 ━━━━━━━━━━━━━━━━━━━━ 33s 292ms/step - accuracy: 0.4855 - loss: 1.7146 - val_accuracy: 0.8160 - val_loss: 0.6382
Epoch 2/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 4s 62ms/step - accuracy: 0.8427 - loss: 0.5278 - val_accuracy: 0.8415 - val_loss: 0.5209
Epoch 3/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 4s 63ms/step - accuracy: 0.8713 - loss: 0.4152 - val_accuracy: 0.8443 - val_loss: 0.4877
Epoch 4/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 5s 65ms/step - accuracy: 0.8953 - loss: 0.3494 - val_accuracy: 0.8616 - val_loss: 0.4593
Epoch 5/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 4s 61ms/step - accuracy: 0.9158 - loss: 0.2945 - val_accuracy: 0.8597 - val_loss: 0.4562
Epoch 6/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 4s 63ms/step - accuracy: 0.9208 - loss: 0.2708 - val_accuracy: 0.8579 - val_loss: 0.4480
Epoch 7/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 4s 61ms/step - accuracy: 0.9377 - loss: 0.2358 - val_accuracy: 0.8552 - val_loss: 0.4489
Epoch 8/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 5s 67ms/step - accuracy: 0.9455 - loss: 0.2114 - val_accuracy: 0.8497 - val_loss: 

**Block 4: Web Deployment & User Interface**  
This block defines the `predict_pest` function to process user uploads and returns the top 3 classified names.  
It launches a Gradio interface with a custom title, subtitle, and article, generating a public URL for real-time agricultural pest recognition.

In [4]:
def predict_pest(img):
    img = np.array(img.resize((224, 224)))
    img = img[np.newaxis, ...] # Add batch dimension
    # No rescaling needed here as it's built into the model above
    preds = model.predict(img)
    return {class_names[i]: float(preds[0][i]) for i in range(len(class_names))}

demo = gr.Interface(
    fn=predict_pest,
    inputs=gr.Image(type="pil"),
    outputs=gr.Label(num_top_classes=3),
    
    # Main large title
    title="Agricultural Pest Classifier",
    
    # Subtitle (appears smaller right below the title)
    description="""### ANN Agricultural Pests Classifier System.
    \n*Upload a clear photo to identify the species and see the confidence score.*""",
    
    # Optional: Add a small note at the very bottom of the app
   article="""
    We developed this model using approximately 6000 realistic agricultural pests data with 12 different types of pests.<br>
    We preferred this data over wine & spirits data coz we were driven to solving some real problems in agricultural sector.
    """
)


demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://2f3af1ca0ad2b1b4b2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
